In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_vendor_master
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
SELECT 
    merchant_code AS vendor_id,
    merchant_name AS vendor_name,
    company_name,
    merchant_description,
    fax,
    email,
    website,
    bank_owner,
    bank_name,
    bank_number,
    bank_branch,
    tax_code,
    block_company_code,
    block_purchasing_org,
    mov
FROM {catalog_name}.udp_wcm_silver_sap_erp.vendor_master
QUALIFY ROW_NUMBER() OVER (PARTITION BY merchant_code ORDER BY proc_date DESC) = 1;
""")